<a href="https://colab.research.google.com/github/alyrazik/News_content_collect_store/blob/MongoDB-Saving-and-querying/CrawlingBBC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  News Content Collect and Store
## by Aly Abdelrazek
 
 


##Setup and Importing libraries

In [114]:
!python -m pip install pymongo[srv]

In [115]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import pickle
import pymongo
from pymongo import MongoClient
from datetime import datetime


In [116]:
pymongo.version

'3.11.0'

##Global variables

In [117]:
news_page_url = "https://bbc.com/news"
absolute_link_prefix = "https://bbc.com"


##Crawl


In [118]:
# Obtains news stories links from news_page_url (https://bbc.com/news)
def news_page_to_news_urls(url, prefix):
    ''' Takes the URL to a news page and the prefix of the absolute link, and returns a list of news stories URLs.'''
    news_urls = []
    news_page_html = requests.get(url).text
    news_page_soup = BeautifulSoup(news_page_html, "html.parser")
    news_page_links = news_page_soup.findAll("a") #find all links
    
    for link in news_page_links:
        href = link.get("href")
        if href.startswith("/news") and href[-1].isdigit(): #news articles start with /news and ends with a digit
            news_url = prefix + href
            s = BeautifulSoup(requests.get(news_url).text , "html.parser")
            if s.find(class_= "css-16rg7hm-ContainerWithSidebarWrapper e1jl38b40") is not None: #this class is only available in news articles
              news_urls.append(news_url)
    return news_urls

news_urls = news_page_to_news_urls(news_page_url, absolute_link_prefix)

In [120]:
len(news_urls)

37

## Scrape

In [ ]:
documents = []
document_index = 0

for url in news_urls:
    print("Fetching {}".format(url))
    response = requests.get(url)
    html = response.text
    soup = BeautifulSoup(html, "html.parser")

    news_heading = (soup.find('h1')).text
    news_text = [p.text for p in soup.find('article').find_all('p')]
    news_time = (soup.find('time'))['datetime']

    documents.append({"Document_Index":str(document_index), "Heading":news_heading, "Text":news_text, "DateTime":news_time})
    document_index = document_index+1


##Saving to MongoDB

In [122]:
#connect to client
client = MongoClient("mongodb+srv://aly:a@cluster0.4pfcp.mongodb.net/db?retryWrites=true&w=majority")

In [75]:
#create a database
db = client["news_database"]

In [123]:
#create a collection (a table)
bbc_news = db["bbc_news"]

In [128]:
db['bbc_news'].delete_many({})

In [129]:
bbc_news.insert_many(documents)

##Retrieving content from MongoDB

In [130]:
def retrieve_documents(database, collection):
  '''takes a MongoDB database and collection name, and returns all documents in collection to a pandas dataframe'''
  retrieved_documents = database[collection].find() #do not use database.collection, it is a string :)
  retrieved_df = pd.DataFrame(retrieved_documents)
  return retrieved_df

db = client["news_database"]
df = retrieve_documents(database = db , collection = 'bbc_news')